# README
This notebook fetches all data required for the risk engine and exports 3 dataframes:
- risk_cust_attributes
- risk_cust_fixed_lists
- risk_cust_trx
  
Note: This notebook will not be turned to a production script (at least inside the risk engine). This shall be replaced by the API call itself either for:
- scoring or rescoring.
- predict mode vs train mode
  
This notebook shall help the BE team to develop the respective DBs and API endpoints.

# 1- Imports

In [8]:
import numpy as np
import pandas as pd
import yaml
import sys
import os
from pathlib import Path
import re

In [6]:
# paths and custom modules
cwd_path = Path(os.getcwd())
data_path = cwd_path / 'data'
sys.path.append(str(cwd_path))
from utils import return_catalog, dataFetchingFunc, strip_strings, stnd_missing_values_type, build_spark_session
data_catalog, models_catalog = return_catalog()

### Export version

In [3]:
data_date = '20240810'
analysis_date = '20240810'

# 2- Fetch

### Filtered train data

In [4]:
query = '''
WITH 
ordered_orders AS (
    SELECT 
        fo.ClientID,
        fo.SSN, 
        fo.ContractDateTime,
        fo.OrderCode,
        ROW_NUMBER() OVER(PARTITION BY fo.ClientID, fo.SSN ORDER BY fo.ContractDateTime ASC) AS row_num
    FROM DWH_MiniCash.dbo.Fact_Orders fo 
    WHERE OrderStatusID IN (8, 12)
),

filtered_orders AS (
    SELECT 
        ClientID,
        SSN,
        ContractDateTime,
        OrderCode
    FROM ordered_orders
    WHERE row_num = 1
),

joined_orders AS (
    SELECT 
        sfo.ClientID, 
        sfo.SSN,
        sfo.ContractDateTime, 
        sfo.OrderCode, 
        fia.RequerdDate, 
        fia.PaymentDate,
        CASE 
            WHEN PaymentDate IS NOT NULL THEN DATEDIFF(DAY, fia.RequerdDate, fia.PaymentDate) 
            ELSE DATEDIFF(DAY, fia.RequerdDate, GETDATE()) 
        END AS DaysDifference,
        CASE 
            WHEN DATEDIFF(DAY, fia.RequerdDate, ISNULL(fia.PaymentDate, GETDATE())) > 90 THEN 1
            ELSE 0
        END AS IsAbove90Days,
        fia.Amount,
        fia.BenefitValue
    FROM filtered_orders sfo
    LEFT JOIN DWH_MiniCash.dbo.Fact_InstallmentAmount fia
    ON sfo.OrderCode = fia.OrderCode
),

grouped_orders AS (
    SELECT 
        jo.ClientID, 
        jo.SSN,
        jo.OrderCode, 
        jo.ContractDateTime, 
        SUM(jo.IsAbove90Days) AS def_flag,
        o.MaxNumberOfPayment AS 'tenor',
        o.OrderID,
        SUM(jo.Amount) AS order_amount
    FROM joined_orders jo
    LEFT JOIN DWH_MiniCash.dbo.Fact_Orders o 
    ON o.ordercode = jo.OrderCode
    GROUP BY jo.ClientID, jo.SSN, jo.ContractDateTime, jo.OrderCode, o.MaxNumberOfPayment, o.OrderID
),

lvl6 AS (
    SELECT 
        SUM(Premium) 'price',
        OrderID 
    FROM DWH_MiniCash.dbo.Fact_OrderDetails 
    WHERE OrderStatusID IN (8,12) 
    GROUP BY OrderID
),

lvl7 AS (
    SELECT 
        SUM(BenefitValue) 'Benfit',
        OrderID 
    FROM DWH_MiniCash.dbo.Fact_OrderDetails 
    WHERE OrderStatusID IN (8,12) 
    GROUP BY OrderID
),

lvl8 AS (
    SELECT 
        SUM(PaymentAmount) 'Total_price',
        OrderID 
    FROM DWH_MiniCash.dbo.Fact_OrderDetails 
    WHERE OrderStatusID IN (8,12) 
    GROUP BY OrderID
)

SELECT      
    go.ClientID, 
    go.SSN, 
    go.ContractDateTime,
    -- ISNULL(go.order_amount,0) AS 'order_amount_t1' ,
    ROUND((SELECT lvl8.Total_price FROM lvl8 WHERE lvl8.orderid = go.orderid) * go.tenor,0) 'order_amount',
    ROUND((SELECT lvl7.Benfit FROM lvl7 WHERE lvl7.orderid = go.orderid) * go.tenor, 0) AS order_benefit,
    go.tenor,
    CASE
        WHEN go.def_flag > 0 THEN 1
        ELSE 0
    END AS def_flag 
FROM grouped_orders go
'''

first_loan_df = dataFetchingFunc(
    Query=query,
    SERVER="BI-DR-DB",
    DATABASE="master"
)

first_loan_df.head()

,ClientID,SSN,ContractDateTime,order_amount,order_benefit,tenor,def_flag
0,3235569,26906052101716,2023-01-29 13:39:58.507,7936.000000,3872.000000,24,0
1,1405151,28409091502314,2019-02-07 12:24:10.600,2798.000000,670.000000,12,0
2,3753420,30503150200764,2024-08-03 21:29:52.970,10752.000000,11604.000000,36,0
3,2083981,26308292201055,2020-02-09 17:13:32.403,6013.000000,3779.000000,36,0
4,635295,26209090103231,2022-06-09 21:44:17.470,8000.000000,1000.000000,6,0


### Income data

In [5]:
query = '''
WITH income_v AS (
    SELECT 
        ClientID, 
        CreateDate,
        ABS(ClientSalary) + ABS(ClientOtherSalary) AS net_income,
        ABS(TotalOtherBurden) + ABS(TotalFamilyBurden) AS net_burden,
        ROW_NUMBER() OVER(PARTITION BY ClientID ORDER BY CreateDate) AS rn,
        ROW_NUMBER() OVER(PARTITION BY ClientID ORDER BY CreateDate DESC) AS rn_desc
    FROM 
        DWH_MiniCash.dbo.Dim_ClientCreditLimit ccl
    WHERE 
        ABS(ClientSalary) + ABS(ClientOtherSalary) > 100 AND
        ABS(ClientSalary) + ABS(ClientOtherSalary) <= 150000
        AND CreateDate >= '2019-01-01'
)
SELECT 
    ClientID,
    MIN(CreateDate) AS first_update_date,
    MAX(CASE WHEN rn = 1 THEN net_income END) AS net_income_first,
    MAX(CASE WHEN rn = 1 THEN net_burden END) AS net_burden_first,
    MAX(CreateDate) AS last_update_date,
    MAX(CASE WHEN rn_desc = 1 THEN net_income END) AS net_income_last,
    MAX(CASE WHEN rn_desc = 1 THEN net_burden END) AS net_burden_last
FROM 
    income_v
GROUP BY 
    ClientID
'''

income_burden_df = dataFetchingFunc(
    Query=query,
    SERVER="BI-DR-DB",
    DATABASE="master"
)

income_burden_df

,ClientID,first_update_date,net_income_first,net_burden_first,last_update_date,net_income_last,net_burden_last
0,4,2020-01-04,1330.0,0.0,2024-02-17,8000.0,0.0
1,42135,2024-07-21,5000.0,0.0,2024-07-24,5000.0,0.0
2,42185,2024-07-21,6000.0,0.0,2024-07-24,6000.0,0.0
3,42201,2019-08-19,4000.0,0.0,2024-03-30,7875.0,0.0
4,42209,2024-07-21,10500.0,0.0,2024-07-24,10500.0,0.0
...,...,...,...,...,...,...,...
1884697,3760768,2024-08-09,9500.0,0.0,2024-08-09,9500.0,0.0
1884698,3760774,2024-08-09,5000.0,0.0,2024-08-09,5000.0,0.0
1884699,3760776,2024-08-09,7000.0,0.0,2024-08-09,7000.0,0.0
1884700,3760800,2024-08-09,8400.0,0.0,2024-08-09,8400.0,0.0


### Application data

In [6]:
query = '''
SELECT oad.ClientNationalNumber AS SSN, ht.HouseTypeName, oad.SportingClubId,
	oad.CarTypeId, oad.CarModel, oad.ChilderenCount 
FROM Lnd_MiniCash.dbo.OrderAdditionalData oad
	INNER JOIN (SELECT ClientNationalNumber, MAX(Id) AS max_id
	FROM Lnd_MiniCash.dbo.OrderAdditionalData oad
	GROUP BY ClientNationalNumber) sub ON sub.max_id = oad.Id 
LEFT JOIN Lnd_MiniCash.dbo.HouseType ht ON oad.HousingTypeId = ht.HouseTypeID
'''

app_d1_df = dataFetchingFunc(
    Query=query,
    SERVER="BI-DR-DB",
    DATABASE="master"
)

app_d1_df

,SSN,HouseTypeName,SportingClubId,CarTypeId,CarModel,ChilderenCount
0,28802122401094,تمليك,NaN,0.0,0.0,2
1,28111140103855,تمليك,NaN,0.0,0.0,2
2,29104201301369,تمليك,NaN,0.0,0.0,3
3,29704098800057,ايجار جديد,NaN,0.0,0.0,0
4,30301060202693,تمليك,NaN,0.0,0.0,0
...,...,...,...,...,...,...
1959352,29009221501159,تمليك,NaN,0.0,0.0,0
1959353,26811060201699,تمليك,NaN,0.0,0.0,0
1959354,28703011818214,تمليك,NaN,0.0,0.0,0
1959355,29501012614711,ايجار جديد,NaN,0.0,0.0,0


In [7]:
query = """
SELECT dc.ClientID, dc.SSN, dc.MobilePhone AS 'phone_number_1',
dc.HomePhone AS 'phone_number_2', dc.JobName, ms.MartialStatusName, dc.ClientClass AS repayment_class
FROM DWH_MINICASH.DBO.DIM_CLIENTS DC
LEFT JOIN LND_MINICASH.DBO.MARTIALSTATUS MS ON MS.MARTIALSTATUSID = DC.MARTIALSTATUSID
ORDER BY DC.CLIENTID DESC
"""

app_d2_df = dataFetchingFunc(
    Query=query,
    SERVER="BI-DR-DB",
    DATABASE="master"
)

app_d2_df

,ClientID,SSN,phone_number_1,phone_number_2,JobName,MartialStatusName,repayment_class
0,3760950,29501023300077,01097827322,01097827322,مهندس,غير متزوج,None
1,3760949,28211060300131,01030057452,01030057452,موظف,None,None
2,3760948,28606210300202,01288985838,01288985838,مهن حرة,متزوج,None
3,3760947,28703130102421,01014173957,01014173957,موظف,متزوج,None
4,3760946,30401142401354,01288820359,01288820359,موظف,غير متزوج,N
...,...,...,...,...,...,...,...
3684465,11,1105080036,None,26427570,N/A,None,A
3684466,10,1105071898,010144633,010144633,N/A,None,N
3684467,8,1105071810,None,26328256,N/A,None,A
3684468,4,1105051517,00000000000,None,N/A,None,N


In [8]:
query = """
SELECT j.jobsName AS job_name, j.EnName AS job_name_english, jt.Name AS job_type, j.RiskId AS job_risk_id
FROM Btech_live.dbo.Jobs j
LEFT JOIN Btech_live.dbo.JobType jt ON j.JobTypeId = jt.Id
"""

app_d3_df = dataFetchingFunc(
    Query=query,
    SERVER="10.3.1.240",
    USERNAME="sa",
    PASSWORD="P@ssw0rd",
    DATABASE="Btech_Live"
)

app_d3_df

,job_name,job_name_english,job_type,job_risk_id
0,طالب,Student,Student,3.0
1,محامى,Lawyer,Crafts,1.0
2,موظف,Employee,Gov,3.0
3,مهندس,Engineer,Prime entities,3.0
4,معاش,Retired,Pension,3.0
...,...,...,...,...
61,مالك عقارات ومحلات,مالك عقارات ومحلات,None,NaN
62,مدرس بمدرسة خاصة,مدرس بمدرسة خاصة,None,NaN
63,مدرس حكومة,مدرس حكومة,None,NaN
64,موظف بغرفة التجارة الامريكية,An employee at the US Chamber of Commerce,Celebrities,2.0


### Address data

In [9]:
# query = """
# WITH max_orders AS (
# SELECT ClientID, MAX(OrderDate) AS max_order_date
# FROM Btech_live.dbo.Orders o 
# GROUP BY ClientID
# )
# SELECT o.ClientID, c.SSN, 
# -- o.OrderID, 
# g.GovernorateAr AS address_governrate, 
# d.DistrictNameAr AS address_city,
# a.AreaNameAr AS address_area
# FROM Btech_live.dbo.Orders o
# INNER JOIN max_orders on (max_orders.ClientID = o.ClientID) AND (max_orders.max_order_date = o.OrderDate)
# LEFT JOIN Btech_live.dbo.Client c ON c.ClientID = o.ClientID 
# LEFT JOIN  Btech_live.dbo.Area a WITH (nolock) on a.AreaID = o.DistrictAreaId
# LEFT JOIN Btech_live.dbo.District d on d.DistrictID = a.DistrictID
# LEFT JOIN Btech_live.dbo.Governorate g on g.GovernID = d.GovernorateID
# LEFT JOIN Btech_live.dbo.OrderFulfillment of2 ON o.OrderID = of2.MCOrderID
# ORDER BY o.ClientID DESC
# """

# cust_address_df = dataFetchingFunc(
#     Query=query,
#     SERVER="10.3.1.240",
#     USERNAME="sa",
#     PASSWORD="P@ssw0rd",
#     DATABASE="Btech_Live"
# )

# cust_address_df

,ClientID,SSN,address_governrate,address_city,address_area
0,3549597,29101118293829,القاهرة,البدرشين,التبين
1,3549596,29101014857466,الجيزة,صعيد مصر اون لاين كاش,صعيد مصر اون لاين كاش
2,3549595,28712140100199,اسوان,ادفو,ادفو المركز
3,3549594,28712140100193,اسوان,ادفو,ادفو المركز
4,3549593,28712140100192,الجيزة,الزمالك,الزمالك
...,...,...,...,...,...
2238461,15,1105080613,None,None,None
2238462,11,1105080036,None,None,None
2238463,10,1105071898,None,None,None
2238464,8,1105071810,None,None,None


In [ ]:
query = """
WITH max_orders AS
  (SELECT ClientID,
          MAX(OrderDate) AS max_order_date
   FROM Btech_live.dbo.Orders o
   GROUP BY ClientID)
SELECT o.ClientID,
       c.SSN,
       -- o.OrderID,
       g.GovernID AS GovernorateID,
       d.DistrictID AS CityID,
       a.AreaID AS AreaID
FROM Btech_live.dbo.Orders o
INNER JOIN max_orders ON (max_orders.ClientID = o.ClientID)
AND (max_orders.max_order_date = o.OrderDate)
LEFT JOIN Btech_live.dbo.Client c ON c.ClientID = o.ClientID
LEFT JOIN Btech_live.dbo.Area a WITH (nolock) ON a.AreaID = o.DistrictAreaId
LEFT JOIN Btech_live.dbo.District d ON d.DistrictID = a.DistrictID
LEFT JOIN Btech_live.dbo.Governorate g ON g.GovernID = d.GovernorateID
LEFT JOIN Btech_live.dbo.OrderFulfillment of2 ON o.OrderID = of2.MCOrderID
ORDER BY o.ClientID DESC
"""

cust_address_df = dataFetchingFunc(
    Query=query,
    SERVER="10.3.1.240",
    USERNAME="sa",
    PASSWORD="P@ssw0rd",
    DATABASE="Btech_Live"
)

In [ ]:
addresss_map = pd.read_excel(data_path+f'feed/live_lists/20240828_mylo_addresses.xlsx', sheet_name='main')

In [ ]:
cust_address_df = cust_address_df.merge(
        addresss_map,
        left_on= ['AreaID', 'CityID','GovernorateID'],
        right_on=['mc_AreaID', 'mc_CityID', 'mc_GovernorateID'],
        how='left',
        indicator=True
    )

cust_address_df = cust_address_df[['ClientID', 'SSN', 'GovernorateEn', 'CityNameEn', 'AreaNameEn']]
cust_address_df.dropna(subset=['GovernorateEn', 'CityNameEn', 'AreaNameEn'], inplace=True)
cust_address_df.rename(columns={'GovernorateEn':'address_governorate',
                        'CityNameEn':'address_city',
                        'AreaNameEn':'address_area'}, inplace=True)

### I-Score data

In [10]:
query = '''
SELECT
    SSN,
    Score AS i_score,
    IscoreReport
FROM Lnd_MiniCash.dbo.IscoreReport
'''

iscore_df = dataFetchingFunc(
    Query=query,
    SERVER="BI-DR-DB",
    DATABASE="master"
)

iscore_df

,SSN,i_score,IscoreReport
0,29310161203333,0.0,"[{""ModuleId"":""iScoreNoHitHeader"",""Content"":{""D..."
1,29209270300095,0.0,"[{""ModuleId"":""iScorePdfIcon"",""Content"":{""DATA""..."
2,26301071200114,0.0,"[{""ModuleId"":""iScorePdfIcon"",""Content"":{""DATA""..."
3,29208010110172,0.0,"[{""ModuleId"":""iScoreNoHitHeader"",""Content"":{""D..."
4,28807171301234,0.0,"[{""ModuleId"":""iScoreNoHitHeader"",""Content"":{""D..."
...,...,...,...
355979,28406072102594,620.0,"[{""ModuleId"":""iScorePdfIcon"",""Content"":{""DATA""..."
355980,27607102103126,749.0,"[{""ModuleId"":""iScorePdfIcon"",""Content"":{""DATA""..."
355981,29401130103824,428.0,"[{""ModuleId"":""iScorePdfIcon"",""Content"":{""DATA""..."
355982,29804030100523,0.0,"[{""ModuleId"":""iScorePdfIcon"",""Content"":{""DATA""..."


### Live lists data

In [13]:
# for b2b customers
b2b_customers_df = pd.read_excel(data_path+'feed/live_lists/20240527_b2b_customer_list.xlsx', dtype=str)[['customer_phone']]
b2b_customers_df = strip_strings(b2b_customers_df)
b2b_cust_list = ', '.join(f"'{phone}'" for phone in b2b_customers_df['customer_phone'].tolist())
b2b_customers_df.head()

,customer_phone
0,01000004992
1,01000007666
2,01000008950
3,01000009022
4,01000011017


In [15]:
# for salary scales (used only to correct current salaries)
salary_scales_df = pd.read_excel(data_path+'feed/live_lists/20240401_salary_scales.xlsx')
salary_scales_df.head()

,ss_job_name,ss_mc_program,ss_min_income,ss_max_income,ss_initial_cl
0,ضابط شرطة,MC02,10000,15000,4000
1,ضابط قوات مسلحة,MC02,10000,25000,5000
2,عضو هيئة قضائية,MC02,20000,35000,6000
3,كبار المسئولين العسكريين ( جيش وشرطة ),MC02,20000,50000,8000
4,كبار المسئولين بالجهات القضائية,MC02,20000,50000,8000


In [16]:
# for mc to mylo job-mapping
mylo_jobs_df = pd.read_excel(data_path+'feed/live_lists/20240620_job_mapping.xlsx')
mylo_jobs_df.head()

,job_name,job_name_map,job_map_min_salary,job_map_max_salary
0,صاحب شركة,Business owner,10000.0,20000.0
1,مكتب مقاولات,Construction & trades,5000.0,10000.0
2,مكاتب المقاولات,Construction & trades,5000.0,10000.0
3,حداد,Construction & trades,5000.0,10000.0
4,مهندس,Engineer,10000.0,30000.0


### Transactions

In [17]:
query = f"""
SELECT [Customer Phone] AS customer_phone,
       [SSoldDateID],
       [Online or  Branch] AS online_vs_branch,
       [Transaction type] AS trx_type,
       [Net Sales] AS net_sales,
       [Quantity] AS net_qty
FROM [DWH_Sales].[dbo].[Fact_online_Sales]
WHERE [Customer Type] = 'StandardCP'
  AND Business_LineValue IN ('B Tech X',
                              'BTECH Mini',
                              'C_CHAINS_MDA',
                              'C_CHAINS_SDA',
                              'C_LED',
                              'C_MDA',
                              'C_SDA',
                              'R_CallCent',
                              'R_Online',
                              'R_Outlet',
                              'R_Stores')
  AND [Customer Phone] NOT IN ({b2b_cust_list})					  
"""

trx_df = dataFetchingFunc(
    Query=query,
    SERVER="BI2-DR-DB",
    DATABASE="master"
)

trx_df

,customer_phone,SSoldDateID,online_vs_branch,trx_type,net_sales,net_qty
0,01005854777,20211229,2,1,43.860,1.000
1,01067750055,20220101,2,1,43.860,1.000
2,01150793081,20220101,2,1,43.860,1.000
3,01112577444,20211230,2,1,43.860,1.000
4,01011981929,20220101,2,1,43.860,1.000
...,...,...,...,...,...,...
9916415,01158658758,20240808,1,1,4297.370,1.000
9916416,01124148283,20240808,2,1,12788.600,1.000
9916417,01124148283,20240808,2,1,8526.320,1.000
9916418,01017900956,20240808,2,1,9209.650,1.000


# 3- Merge & Formatting

In [18]:
# Strip all strings
first_loan_df = strip_strings(first_loan_df)
income_burden_df = strip_strings(income_burden_df)
salary_scales_df = strip_strings(salary_scales_df)
mylo_jobs_df = strip_strings(mylo_jobs_df)
iscore_df = strip_strings(iscore_df)
app_d1_df = strip_strings(app_d1_df)
app_d2_df = strip_strings(app_d2_df)
app_d3_df = strip_strings(app_d3_df)
cust_address_df = strip_strings(cust_address_df)
trx_df = strip_strings(trx_df)

In [19]:
# Merge customer's attributes
cust_attr_df = first_loan_df.merge(income_burden_df, how='left', on='ClientID', indicator='indc_income_burden_df')\
    .merge(iscore_df, how='left', on='SSN', indicator='indc_iscore_df')\
        .merge(app_d1_df, how='left', on='SSN', indicator='indc_app_d1_df')\
            .merge(app_d2_df, how='left', on='ClientID', indicator='indc_app_d2_df')\
                .merge(app_d3_df, how='left', left_on='JobName', right_on='job_name', indicator='indc_app_d3_df')\
                    .merge(salary_scales_df, how='left', left_on='JobName', right_on='ss_job_name', indicator='indc_salary_scales')\
                        .merge(cust_address_df, on='ClientID', how='left', indicator='indc_cust_address_df')\
                            .merge(mylo_jobs_df, on='job_name', how='left', indicator='indc_mylo_jobs_df')
   
 
cust_attr_df.head()

,ClientID,SSN_x,ContractDateTime,order_amount,order_benefit,tenor,def_flag,first_update_date,net_income_first,net_burden_first,...,indc_salary_scales,SSN,address_governrate,address_city,address_area,indc_cust_address_df,job_name_map,job_map_min_salary,job_map_max_salary,indc_mylo_jobs_df
0,3235569,26906052101716,2023-01-29 13:39:58.507,7936.000000,3872.000000,24,0,2023-01-29,1000.0,0.0,...,both,26906052101716,القاهرة,امبابة,ارض الحداد,both,"Other, employed",9166.666667,24166.666667,both
1,1405151,28409091502314,2019-02-07 12:24:10.600,2798.000000,670.000000,12,0,2022-12-13,3200.0,0.0,...,both,28409091502314,كفر الشيخ,دسوق,دسوق,both,"Other, employed",9166.666667,24166.666667,both
2,3753420,30503150200764,2024-08-03 21:29:52.970,10752.000000,11604.000000,36,0,2024-08-03,1250.0,0.0,...,both,NaN,NaN,NaN,NaN,left_only,Student,2500.000000,2500.000000,both
3,2083981,26308292201055,2020-02-09 17:13:32.403,6013.000000,3779.000000,36,0,2020-02-09,2700.0,0.0,...,both,26308292201055,None,None,None,both,"Other, employed",9166.666667,24166.666667,both
4,635295,26209090103231,2022-06-09 21:44:17.470,8000.000000,1000.000000,6,0,2022-06-09,10239.0,0.0,...,both,26209090103231,الاسكندرية,العجمي,بيطاش,both,Retired,3750.000000,7250.000000,both


In [20]:
# def-sequential cleaning (trx)
def base_clean_for_train_trx(trx_df):
    
    trx_df = trx_df.rename(columns={'SSoldDateID': 'trx_date'})
    trx_df = trx_df.rename(columns={'trx_type': 'sales_vs_returns'})
    
    trx_df = trx_df.astype({'net_sales': 'float64'})
    trx_df = trx_df.astype({'net_qty': 'float64'})
    trx_df['trx_date'] = pd.to_datetime(trx_df['trx_date'], format='%Y%m%d')
    
    return trx_df

trx_df_l2 = base_clean_for_train_trx(trx_df.copy())
trx_df_l2.head()

,customer_phone,trx_date,online_vs_branch,sales_vs_returns,net_sales,net_qty
0,01005854777,2021-12-29,2,1,43.86,1.0
1,01067750055,2022-01-01,2,1,43.86,1.0
2,01150793081,2022-01-01,2,1,43.86,1.0
3,01112577444,2021-12-30,2,1,43.86,1.0
4,01011981929,2022-01-01,2,1,43.86,1.0


In [21]:
# def-sequential cleaning (attributes)

def base_clean_for_train_attr(df):

    df = df.rename(columns={'ClientID': 'client_id'})
    df = df.drop_duplicates(subset=['client_id'])
    df = df.rename(columns={'SSN_x': 'ssn'})
    df = df.rename(columns={'ContractDateTime': 'contract_date'})
    df = df.rename(columns={'tenor': 'first_ord_tenor'})
    df = df.dropna(subset=['order_amount'])
    df = df.rename(columns={'order_amount': 'first_ord_amount'})
    df = df.rename(columns={'order_benefit': 'first_ord_benefit'})
    df = df.dropna(subset=['contract_date'])
    df = df.astype({'first_ord_amount': 'float64'})
    df = df.astype({'first_ord_benefit': 'float64'})
    df = df.dropna(subset=['first_update_date'])
    df = df.astype({'first_update_date': 'datetime64[ns]'})
    df = df.rename(columns={'first_update_date': 'first_income_up_date'})
    df = df.rename(columns={'last_update_date': 'last_income_up_date'})
    df = df.astype({'last_income_up_date': 'datetime64[ns]'})
    df = df.drop(columns=['indc_income_burden_df'])
    df = df.rename(columns={'def_flag': 'fo_par90_flag'})
    df = df.rename(columns={'i_score': 'iscore_score'})
    df = df.rename(columns={'IscoreReport': 'iscore_report'})
    df = df.rename(columns={'is_iScoreNohitConsumer': 'is_iscore_nohit'})
    df = df.rename(columns={'HouseTypeName': 'house_type'})
    df = df.rename(columns={'SportingClubId': 'club_level'})
    df = df[df['indc_app_d1_df'].astype("string") == "both"]
    df = df.rename(columns={'CarTypeId': 'car_type_id'})
    df = df.rename(columns={'CarModel': 'car_model_year'})
    df = df.rename(columns={'ChilderenCount': 'children_count'})
    df = df.drop(columns=['indc_app_d1_df', 'SSN_y'])
    df = df.drop(columns=['JobName'])
    df = df.rename(columns={'MartialStatusName': 'marital_status'})
    df = df.drop(columns=['indc_app_d2_df'])
    df = df.drop(columns=['indc_app_d3_df'])
    df = df.drop(columns=['indc_salary_scales'])
    
    df['contract_date'] = pd.to_datetime(df['contract_date']).dt.date 
    df = df.astype({'contract_date': 'datetime64[ns]'})
    df['last_income_up_date'] = pd.to_datetime(df['last_income_up_date']).dt.date
    df = df.astype({'last_income_up_date': 'datetime64[ns]'})
    df['club_level'] = df['club_level'].astype('float64').astype('Int64')
    df['car_type_id'] = df['car_type_id'].astype('float64').astype('Int64')
    df['car_model_year'] = df['car_model_year'].astype('float64').astype('Int64')
    df['children_count'] = df['children_count'].astype('float64').astype('Int64')
    df = df.rename(columns={'repayment_class': 'current_repayment_class'})
    df = df.drop(columns=['ss_job_name'])
    
    return df

cust_attr_df_l2 = base_clean_for_train_attr(cust_attr_df.copy())
cust_attr_df_l2.head()

,client_id,ssn,contract_date,first_ord_amount,first_ord_benefit,first_ord_tenor,fo_par90_flag,first_income_up_date,net_income_first,net_burden_first,...,ss_initial_cl,SSN,address_governrate,address_city,address_area,indc_cust_address_df,job_name_map,job_map_min_salary,job_map_max_salary,indc_mylo_jobs_df
0,3235569,26906052101716,2023-01-29,7936.0,3872.0,24,0,2023-01-29,1000.0,0.0,...,2500.0,26906052101716,القاهرة,امبابة,ارض الحداد,both,"Other, employed",9166.666667,24166.666667,both
1,1405151,28409091502314,2019-02-07,2798.0,670.0,12,0,2022-12-13,3200.0,0.0,...,2500.0,28409091502314,كفر الشيخ,دسوق,دسوق,both,"Other, employed",9166.666667,24166.666667,both
2,3753420,30503150200764,2024-08-03,10752.0,11604.0,36,0,2024-08-03,1250.0,0.0,...,1000.0,NaN,NaN,NaN,NaN,left_only,Student,2500.000000,2500.000000,both
3,2083981,26308292201055,2020-02-09,6013.0,3779.0,36,0,2020-02-09,2700.0,0.0,...,2500.0,26308292201055,None,None,None,both,"Other, employed",9166.666667,24166.666667,both
4,635295,26209090103231,2022-06-09,8000.0,1000.0,6,0,2022-06-09,10239.0,0.0,...,1500.0,26209090103231,الاسكندرية,العجمي,بيطاش,both,Retired,3750.000000,7250.000000,both


In [22]:
# view after def-sequential cleaning
print(cust_attr_df_l2.shape)
print(cust_attr_df_l2['fo_par90_flag'].value_counts(normalize=True))

(797606, 42)
0    0.930862
1    0.069138
Name: fo_par90_flag, dtype: float64


# 4- Flags and other attributes

In [23]:
# std copy
cust_attr_df_lf = cust_attr_df_l2.copy()

# adding flags
cust_attr_df_lf['flag_is_mc_customer'] = 1
cust_attr_df_lf['flag_is_prv_cash_trx'] = 0
cust_attr_df_lf['flag_is_rescore'] = 0

# adding dummy address atributes
# cust_attr_df_lf['address_governorate'] = 'default_value'
# cust_attr_df_lf['address_city'] = 'default_value'
# cust_attr_df_lf['address_area'] = 'default_value'

# adding other attributes
cust_attr_df_lf['insurance_type'] = 'default_value'
cust_attr_df_lf['mobile_os_type'] = 'default_value'

cust_attr_df_lf['net_income'] = cust_attr_df_lf['net_income_last']
cust_attr_df_lf['net_burden'] = cust_attr_df_lf['net_burden_last']

In [24]:
# re-ordering columns for readability
cust_attr_df_lf = cust_attr_df_lf[['client_id', 'ssn', 'phone_number_1', 'phone_number_2','flag_is_mc_customer', 
'flag_is_prv_cash_trx', 'flag_is_rescore', 'contract_date',
#'job_name',
'job_name_map', 
'job_type','net_income', 'net_burden','first_income_up_date','net_income_first', 'net_burden_first', 'last_income_up_date',
'net_income_last', 'net_burden_last',
'ss_mc_program', 'ss_min_income', 'ss_max_income', 'ss_initial_cl',
'job_map_min_salary',
'job_map_max_salary',
'insurance_type', 'marital_status', 'children_count', 'address_governorate', 'address_city', 'address_area',
'house_type', 'car_type_id', 'car_model_year', 'club_level', 'mobile_os_type', 'indc_iscore_df',
'iscore_score', 'iscore_report', 'current_repayment_class', 'first_ord_amount',  'first_ord_benefit','first_ord_tenor','fo_par90_flag']].copy()

# 5- EOD Formatting

In [25]:
def clean_eod_data_att(df):
    df = df.astype({'flag_is_mc_customer': 'int8'})
    df = df.astype({'flag_is_prv_cash_trx': 'int8', 'flag_is_rescore': 'int8'})
    df = df.astype({'net_income_first': 'float32', 'net_burden_first': 'float32', 'net_income_last': 'float32', 'net_burden_last': 'float32',  'net_income': 'float32', 'net_burden': 'float32'})
    df = df.astype({'ss_min_income': 'float32', 'ss_max_income': 'float32', 'ss_initial_cl': 'float32'})
    df = df.astype({'first_ord_amount': 'float32', 'first_ord_benefit': 'float32'})
    df = df.astype({'children_count': 'int8'})
    df = df.astype({'car_type_id': 'string'})
    df = df.astype({'iscore_score': 'float32'})
    df = df.astype({'first_ord_amount': 'float32'})
    df = df.astype({'first_ord_tenor': 'int16'})
    df = df.astype({'fo_par90_flag': 'int8'})
    df = df.astype({'phone_number_1': 'string', 'phone_number_2': 'string', 'club_level': 'string'})
    df = df.astype({'indc_iscore_df': 'string'})
    df['indc_iscore_df'] = df['indc_iscore_df'].str.replace("both", "1", case=False, regex=False)
    df['indc_iscore_df'] = df['indc_iscore_df'].str.replace("left_only", "0", case=False, regex=False)
    df = df.rename(columns={'indc_iscore_df': 'is_iscore'})
    df = df.astype({'is_iscore': 'int8'})
    
    df.loc[df['car_model_year'] == 0, 'car_model_year'] = np.NaN
    df['car_model_year'] = pd.to_datetime(df['car_model_year'], format='%Y', errors='coerce')
    df['car_model_year'] = pd.to_datetime(df['car_model_year'], format='%Y%m%d')
    df['job_map_min_salary'] = df['job_map_min_salary'].round(0)
    df['job_map_max_salary'] = df['job_map_max_salary'].round(0)
    return df

cust_attr_df_eod = clean_eod_data_att(cust_attr_df_lf.copy())
cust_attr_df_eod.head()

,client_id,ssn,phone_number_1,phone_number_2,flag_is_mc_customer,flag_is_prv_cash_trx,flag_is_rescore,contract_date,job_name_map,job_type,...,club_level,mobile_os_type,is_iscore,iscore_score,iscore_report,current_repayment_class,first_ord_amount,first_ord_benefit,first_ord_tenor,fo_par90_flag
0,3235569,26906052101716,01205040486,01104646590,1,0,0,2023-01-29,"Other, employed",Gov,...,<NA>,default_value,0,NaN,NaN,A,7936.0,3872.0,24,0
1,1405151,28409091502314,01016635020,01092085940,1,0,0,2019-02-07,"Other, employed",Gov,...,<NA>,default_value,0,NaN,NaN,C,2798.0,670.0,12,0
2,3753420,30503150200764,01220955775,01224328234,1,0,0,2024-08-03,Student,Student,...,<NA>,default_value,1,0.0,"[{""ModuleId"":""iScoreNoHitHeader"",""Content"":{""D...",N1,10752.0,11604.0,36,0
3,2083981,26308292201055,01271054735,01271054735,1,0,0,2020-02-09,"Other, employed",Gov,...,<NA>,default_value,0,NaN,NaN,A,6013.0,3779.0,36,0
4,635295,26209090103231,01004014651,01002550651,1,0,0,2022-06-09,Retired,Pension,...,<NA>,default_value,1,772.0,"[{""ModuleId"":""iScorePdfIcon"",""Content"":{""DATA""...",A,8000.0,1000.0,6,0


In [26]:
def clean_eod_data_trx(df):
    df = df.astype({'online_vs_branch': 'int8'})
    df = df.astype({'sales_vs_returns': 'int8'})
    df = df.astype({'net_sales': 'float32'})
    df = df.astype({'net_qty': 'float32'})
    return df

cust_trx_df_eod = clean_eod_data_trx(trx_df_l2.copy())
cust_trx_df_eod.head()

,customer_phone,trx_date,online_vs_branch,sales_vs_returns,net_sales,net_qty
0,01005854777,2021-12-29,2,1,43.860001,1.0
1,01067750055,2022-01-01,2,1,43.860001,1.0
2,01150793081,2022-01-01,2,1,43.860001,1.0
3,01112577444,2021-12-30,2,1,43.860001,1.0
4,01011981929,2022-01-01,2,1,43.860001,1.0


In [27]:
def convert_int8_to_int16(df):
    for col in df.columns:
        if df[col].dtype == 'Int8':
            df[col] = df[col].astype('Int16')
    return df

cust_attr_df_eod = convert_int8_to_int16(cust_attr_df_eod.copy())
cust_trx_df_eod = convert_int8_to_int16(cust_trx_df_eod.copy())

In [ ]:
# Update Marital Status to English
mapping = {
    'متزوج': 'Married',
    'غير متزوج': 'Single',
    'مطلق': 'Divorced',
    'ارمل':'Widowed'
}
cust_attr_df_eod['marital_status'] = cust_attr_df_eod['marital_status'].replace(mapping)

# Update House Type to English
mapping = {
    'ايجار جديد': 'New Rent',
    'تمليك': 'Owned',
    'ايجار قديم': 'Old Rent'
}
cust_attr_df_eod['house_type'] = cust_attr_df_eod['house_type'].replace(mapping)

In [12]:
# standardize missing values
cust_attr_df_eod = stnd_missing_values_type(cust_attr_df_eod.copy())
cust_trx_df_eod = stnd_missing_values_type(cust_trx_df_eod.copy())

# 6- Export

--------------

In [29]:
cust_attr_df_eod.dtypes

client_id                           int64
ssn                                object
phone_number_1                     string
phone_number_2                     string
flag_is_mc_customer                  int8
flag_is_prv_cash_trx                 int8
flag_is_rescore                      int8
contract_date              datetime64[ns]
job_name_map                       object
job_type                           object
net_income                        float32
net_burden                        float32
first_income_up_date       datetime64[ns]
net_income_first                  float32
net_burden_first                  float32
last_income_up_date        datetime64[ns]
net_income_last                   float32
net_burden_last                   float32
ss_mc_program                      object
ss_min_income                     float32
ss_max_income                     float32
ss_initial_cl                     float32
job_map_min_salary                float64
job_map_max_salary                

In [30]:
cust_trx_df_eod.dtypes

customer_phone              object
trx_date            datetime64[ns]
online_vs_branch              int8
sales_vs_returns              int8
net_sales                  float32
net_qty                    float32
dtype: object

In [14]:
# Save to_model customer's attributes
cust_attr_df_eod.to_parquet(data_path/f'feed/{data_date}_risk_cust_attributes.parquet')
# Save to_model customer's trx
# cust_trx_df_eod.to_parquet(data_path+f'feed/{data_date}_risk_cust_trx.parquet')

# Analytics/fetch

In [32]:
# query = '''
# WITH RankedInquiries AS (
#     SELECT 
#         ClientID,
#         SSN,
#         [Last Inquiry Date],
#         Class,
#         ROUND(CreditLimit, 0) AS CreditLimit,
#         ROUND(MaxCredit, 0) AS MaxCredit,
#         ROUND(Amount, 0) AS Amount,
#         ROW_NUMBER() OVER (PARTITION BY SSN ORDER BY [Last Inquiry Date] DESC) AS rn
#     FROM 
#         Lnd_MiniCash.dbo.ClinetClass2020
# ),
# RankedAmounts AS (
#     SELECT
#         ClientID,
#         ROUND(MaxAmount, 0) AS MaxAmount,
#         CreateDate,
#         ROW_NUMBER() OVER (PARTITION BY ClientID ORDER BY CreateDate DESC) AS rn
#     FROM
#         Lnd_MiniCash.dbo.ClientCreditLimit
# ),
# FinalResults AS (
#     SELECT 
#         ri.SSN AS ssn,
#         ri.ClientID AS client_id,
#         ri.[Last Inquiry Date] AS inquiry_date,
#         ri.Class AS mc_repayment_class,
#         ri.MaxCredit AS mc_monthly_cl,
#         CASE
#             WHEN Class = 'A' THEN 1.5
#             WHEN Class IN ('B', 'B1') THEN 1.25
#             WHEN Class IN ('N', 'N1', 'N3', 'N4') THEN 1
#             WHEN Class IN ('C', 'C1', 'N2') THEN 0
#             ELSE NULL -- Optional: in case of unexpected class values
#         END AS mc_cl_rate,
#         c.mylo_migrated AS is_mylo_migrated,
#         ra.max_monthly_cl_pre_last,
#         ra.max_monthly_cl_last
#     FROM 
#         RankedInquiries AS ri
#     LEFT JOIN 
#         Lnd_MiniCash.dbo.Client c ON c.ClientID = ri.ClientID
#     LEFT JOIN (
#         SELECT
#             ClientID,
#             MAX(CASE WHEN rn = 2 THEN MaxAmount END) AS max_monthly_cl_pre_last,
#             MAX(CASE WHEN rn = 1 THEN MaxAmount END) AS max_monthly_cl_last
#         FROM
#             RankedAmounts
#         GROUP BY
#             ClientID
#     ) AS ra ON ra.ClientID = ri.ClientID
#     WHERE 
#         ri.rn = 1
# )
# SELECT 
#     ssn,
#     client_id,
#     inquiry_date,
#     mc_repayment_class,
#     ROUND(mc_monthly_cl, 0) AS mc_monthly_cl,
#     mc_cl_rate,
#     is_mylo_migrated,
#     ROUND(
#         CASE 
#             WHEN ROUND(mc_monthly_cl, 0) >= ROUND(max_monthly_cl_pre_last, 0) AND ROUND(mc_monthly_cl, 0) >= ROUND(max_monthly_cl_last, 0) THEN ROUND(mc_monthly_cl, 0)
#             WHEN ROUND(max_monthly_cl_pre_last, 0) >= ROUND(mc_monthly_cl, 0) AND ROUND(max_monthly_cl_pre_last, 0) >= ROUND(max_monthly_cl_last, 0) THEN ROUND(max_monthly_cl_pre_last, 0)
#             ELSE ROUND(max_monthly_cl_last, 0)
#         END, 0
#     ) AS mc_cl_migrated,
#     ROUND(
#         CASE 
#             WHEN CASE 
#                     WHEN ROUND(mc_monthly_cl, 0) >= ROUND(max_monthly_cl_pre_last, 0) AND ROUND(mc_monthly_cl, 0) >= ROUND(max_monthly_cl_last, 0) THEN ROUND(mc_monthly_cl, 0)
#                     WHEN ROUND(max_monthly_cl_pre_last, 0) >= ROUND(mc_monthly_cl, 0) AND ROUND(max_monthly_cl_pre_last, 0) >= ROUND(max_monthly_cl_last, 0) THEN ROUND(max_monthly_cl_pre_last, 0)
#                     ELSE ROUND(max_monthly_cl_last, 0)
#                 END IN (0, 1) THEN NULL
#             ELSE CASE 
#                     WHEN ROUND(mc_monthly_cl, 0) >= ROUND(max_monthly_cl_pre_last, 0) AND ROUND(mc_monthly_cl, 0) >= ROUND(max_monthly_cl_last, 0) THEN ROUND(mc_monthly_cl, 0)
#                     WHEN ROUND(max_monthly_cl_pre_last, 0) >= ROUND(mc_monthly_cl, 0) AND ROUND(max_monthly_cl_pre_last, 0) >= ROUND(max_monthly_cl_last, 0) THEN ROUND(max_monthly_cl_pre_last, 0)
#                     ELSE ROUND(max_monthly_cl_last, 0)
#                 END * mc_cl_rate * 18
#         END, 0
#     ) AS mylo_rev18_cl
# FROM 
#     FinalResults;
# '''

# cl_migration_df = dataFetchingFunc(
#     Query=query,
#     SERVER="BI-DR-DB",
#     DATABASE="master"
# )

# cl_migration_df.head()

,ssn,client_id,inquiry_date,mc_repayment_class,mc_monthly_cl,mc_cl_rate,is_mylo_migrated,mc_cl_migrated,mylo_rev18_cl
0,1097080251,8112,NaT,B1,0.0,1.25,False,NaN,NaN
1,1097080401,20756,NaT,A,0.0,1.50,False,NaN,NaN
2,1097080661,17982,NaT,A,0.0,1.50,False,NaN,NaN
3,1097090066,20798,NaT,A,0.0,1.50,False,NaN,NaN
4,1097090259,37800,NaT,A,0.0,1.50,False,NaN,NaN


In [ ]:
query = '''
WITH RankedInquiries AS (
    SELECT 
        ClientID,
        SSN,
        [Last Inquiry Date],
        Class,
        CreditLimit,
        MaxCredit,
        Amount,
        ROW_NUMBER() OVER (PARTITION BY SSN ORDER BY [Last Inquiry Date] DESC) AS rn
    FROM 
        Lnd_MiniCash.dbo.ClinetClass2020
),
RankedAmounts AS (
    SELECT
        ClientID,
        MaxAmount,
        CreateDate,
        ROW_NUMBER() OVER (PARTITION BY ClientID ORDER BY CreateDate DESC) AS rn
    FROM
        Lnd_MiniCash.dbo.ClientCreditLimit
),
FinalResults AS (
    SELECT 
        ri.SSN AS ssn,
        ri.ClientID AS client_id,
        ri.[Last Inquiry Date] AS inquiry_date,
        ri.Class AS mc_repayment_class,
        ri.MaxCredit AS mc_monthly_cl,
        CASE
            WHEN Class = 'A' THEN 1.5
            WHEN Class IN ('B', 'B1') THEN 1.25
            WHEN Class IN ('N', 'N1', 'N3', 'N4') THEN 1
            WHEN Class IN ('C', 'C1', 'N2') THEN 0
            ELSE NULL
        END AS mc_cl_rate,
        c.mylo_migrated AS is_mylo_migrated,
        ra.max_monthly_cl_pre_last,
        ra.max_monthly_cl_last
    FROM 
        RankedInquiries AS ri
    LEFT JOIN 
        Lnd_MiniCash.dbo.Client c ON c.ClientID = ri.ClientID
    LEFT JOIN (
        SELECT
            ClientID,
            MAX(CASE WHEN rn = 2 THEN MaxAmount END) AS max_monthly_cl_pre_last,
            MAX(CASE WHEN rn = 1 THEN MaxAmount END) AS max_monthly_cl_last
        FROM
            RankedAmounts
        GROUP BY
            ClientID
    ) AS ra ON ra.ClientID = ri.ClientID
    WHERE 
        ri.rn = 1
)
SELECT 
    ssn,
    client_id,
    inquiry_date,
    mc_repayment_class,
    mc_cl_rate,
    is_mylo_migrated,
    ROUND(
        CASE 
            WHEN max_monthly_cl_last > 0 THEN max_monthly_cl_last
            WHEN max_monthly_cl_pre_last > 0 THEN max_monthly_cl_pre_last
            ELSE mc_monthly_cl
        END, 0
    ) AS mc_cl_migrated,
    ROUND(
        CASE 
            WHEN max_monthly_cl_last > 0 THEN max_monthly_cl_last * mc_cl_rate * 18
            WHEN max_monthly_cl_pre_last > 0 THEN max_monthly_cl_pre_last * mc_cl_rate * 18
            ELSE mc_monthly_cl * mc_cl_rate * 18
        END, 0
    ) AS mylo_rev18_cl
FROM 
    FinalResults;
'''

cl_migration_df = dataFetchingFunc(
    Query=query,
    SERVER="BI-DR-DB",
    DATABASE="master"
)

cl_migration_df = cl_migration_df.astype({'mc_cl_rate': 'float64'})
cl_migration_df = cl_migration_df.astype({'is_mylo_migrated': 'bool'})

cl_migration_df.head()

In [34]:
cl_migration_df.to_parquet(data_path+f'analytics/{analysis_date}_cl_migration_df.parquet')